In [43]:
# pyright (c) Facebook, Inc. and its affiliates.
import re

from tqdm import tqdm


class EvalAIAnswerProcessor:
    """
    Processes an answer similar to Eval AI
        copied from
        https://github.com/facebookresearch/mmf/blob/c46b3b3391275b4181567db80943473a89ab98ab/pythia/tasks/processors.py#L897
    """

    CONTRACTIONS = {
        "aint": "ain't",
        "arent": "aren't",
        "cant": "can't",
        "couldve": "could've",
        "couldnt": "couldn't",
        "couldn'tve": "couldn't've",
        "couldnt've": "couldn't've",
        "didnt": "didn't",
        "doesnt": "doesn't",
        "dont": "don't",
        "hadnt": "hadn't",
        "hadnt've": "hadn't've",
        "hadn'tve": "hadn't've",
        "hasnt": "hasn't",
        "havent": "haven't",
        "hed": "he'd",
        "hed've": "he'd've",
        "he'dve": "he'd've",
        "hes": "he's",
        "howd": "how'd",
        "howll": "how'll",
        "hows": "how's",
        "Id've": "I'd've",
        "I'dve": "I'd've",
        "Im": "I'm",
        "Ive": "I've",
        "isnt": "isn't",
        "itd": "it'd",
        "itd've": "it'd've",
        "it'dve": "it'd've",
        "itll": "it'll",
        "let's": "let's",
        "maam": "ma'am",
        "mightnt": "mightn't",
        "mightnt've": "mightn't've",
        "mightn'tve": "mightn't've",
        "mightve": "might've",
        "mustnt": "mustn't",
        "mustve": "must've",
        "neednt": "needn't",
        "notve": "not've",
        "oclock": "o'clock",
        "oughtnt": "oughtn't",
        "ow's'at": "'ow's'at",
        "'ows'at": "'ow's'at",
        "'ow'sat": "'ow's'at",
        "shant": "shan't",
        "shed've": "she'd've",
        "she'dve": "she'd've",
        "she's": "she's",
        "shouldve": "should've",
        "shouldnt": "shouldn't",
        "shouldnt've": "shouldn't've",
        "shouldn'tve": "shouldn't've",
        "somebody'd": "somebodyd",
        "somebodyd've": "somebody'd've",
        "somebody'dve": "somebody'd've",
        "somebodyll": "somebody'll",
        "somebodys": "somebody's",
        "someoned": "someone'd",
        "someoned've": "someone'd've",
        "someone'dve": "someone'd've",
        "someonell": "someone'll",
        "someones": "someone's",
        "somethingd": "something'd",
        "somethingd've": "something'd've",
        "something'dve": "something'd've",
        "somethingll": "something'll",
        "thats": "that's",
        "thered": "there'd",
        "thered've": "there'd've",
        "there'dve": "there'd've",
        "therere": "there're",
        "theres": "there's",
        "theyd": "they'd",
        "theyd've": "they'd've",
        "they'dve": "they'd've",
        "theyll": "they'll",
        "theyre": "they're",
        "theyve": "they've",
        "twas": "'twas",
        "wasnt": "wasn't",
        "wed've": "we'd've",
        "we'dve": "we'd've",
        "weve": "we've",
        "werent": "weren't",
        "whatll": "what'll",
        "whatre": "what're",
        "whats": "what's",
        "whatve": "what've",
        "whens": "when's",
        "whered": "where'd",
        "wheres": "where's",
        "whereve": "where've",
        "whod": "who'd",
        "whod've": "who'd've",
        "who'dve": "who'd've",
        "wholl": "who'll",
        "whos": "who's",
        "whove": "who've",
        "whyll": "why'll",
        "whyre": "why're",
        "whys": "why's",
        "wont": "won't",
        "wouldve": "would've",
        "wouldnt": "wouldn't",
        "wouldnt've": "wouldn't've",
        "wouldn'tve": "wouldn't've",
        "yall": "y'all",
        "yall'll": "y'all'll",
        "y'allll": "y'all'll",
        "yall'd've": "y'all'd've",
        "y'alld've": "y'all'd've",
        "y'all'dve": "y'all'd've",
        "youd": "you'd",
        "youd've": "you'd've",
        "you'dve": "you'd've",
        "youll": "you'll",
        "youre": "you're",
        "youve": "you've",
    }

    NUMBER_MAP = {
        "none": "0",
        "zero": "0",
        "one": "1",
        "two": "2",
        "three": "3",
        "four": "4",
        "five": "5",
        "six": "6",
        "seven": "7",
        "eight": "8",
        "nine": "9",
        "ten": "10",
    }
    ARTICLES = ["a", "an", "the"]
    PERIOD_STRIP = re.compile(r"(?!<=\d)(\.)(?!\d)")
    # text = "Version 3.0 is available. The price is 10.50 dollars." -> Version 3.0 is available The price is 10.50 dollars
    # 이 정규 표현식은 숫자 사이에 있지 않은 마침표를 찾습니다.

    COMMA_STRIP = re.compile(r"(?<=\d)(\,)+(?=\d)")
    # 이 정규 표현식은 숫자 사이에 있는 쉼표를 찾고, 이를 제거합니다.
    PUNCTUATIONS = [
        ";",
        r"/",
        "[",
        "]",
        '"',
        "{",
        "}",
        "(",
        ")",
        "=",
        "+",
        "\\",
        "_",
        "-",
        ">",
        "<",
        "@",
        "`",
        ",",
        "?",
        "!",
    ]

    def __init__(self, *args, **kwargs):
        pass

    def word_tokenize(self, word):
        word = word.lower()
        word = word.replace(",", "").replace("?", "").replace("'s", " 's")
        return word.strip()

    def process_punctuation(self, in_text):
        out_text = in_text
        for p in self.PUNCTUATIONS:
            if (p + " " in in_text or " " + p in in_text) or (
                re.search(self.COMMA_STRIP, in_text) is not None
            ):
                out_text = out_text.replace(p, "")
            else:
                out_text = out_text.replace(p, " ")
        out_text = self.PERIOD_STRIP.sub("", out_text, re.UNICODE)
        return out_text

    def process_digit_article(self, in_text):
        out_text = []
        temp_text = in_text.lower().split()
        for word in temp_text:
            word = self.NUMBER_MAP.setdefault(word, word)
            if word not in self.ARTICLES:
                out_text.append(word)
            else:
                pass
        for word_id, word in enumerate(out_text):
            if word in self.CONTRACTIONS:
                out_text[word_id] = self.CONTRACTIONS[word]
        out_text = " ".join(out_text)
        return out_text

    def __call__(self, item):
        item = self.word_tokenize(item)
        item = item.replace("\n", " ").replace("\t", " ").strip()
        item = self.process_punctuation(item)
        item = self.process_digit_article(item)
        return item


class TextVQAAccuracyEvaluator:
    def __init__(self):
        self.answer_processor = EvalAIAnswerProcessor()

    def _compute_answer_scores(self, raw_answers):
        """
        compute the accuracy (soft score) of human answers
        """
        answers = [self.answer_processor(a) for a in raw_answers]
        assert len(answers) == 10
        gt_answers = list(enumerate(answers))
        unique_answers = set(answers)
        unique_answer_scores = {}

        for unique_answer in unique_answers:
            accs = []
            for gt_answer in gt_answers:
                other_answers = [item for item in gt_answers if item != gt_answer]
                matching_answers = [
                    item for item in other_answers if item[1] == unique_answer
                ]
                acc = min(1, float(len(matching_answers)) / 3)
                accs.append(acc)
            unique_answer_scores[unique_answer] = sum(accs) / len(accs)

        return unique_answer_scores

    def eval_pred_list(self, pred_list):
        pred_scores = []
        for entry in tqdm(pred_list):
            pred_answer = self.answer_processor(entry["pred_answer"])
            unique_answer_scores = self._compute_answer_scores(entry["gt_answers"])
            score = unique_answer_scores.get(pred_answer, 0.0)
            pred_scores.append(score)

        accuracy = sum(pred_scores) / len(pred_scores)
        return accuracy


In [ ]:
# prompt: textvqa.json 파일 열어줘

import json

with open('val.json', 'r') as f:
  data = json.load(f)

data = data['data'][:50]



In [35]:
data = data['data'][:50]

In [54]:
dataset = [(data['flickr_300k_url'], data['question'].lower(), data['answers']) for data in data['data'][:50]]


In [71]:
dataset[3]

('https://c3.staticflickr.com/1/559/18386954540_abe56a93cf_z.jpg',
 'what does the light sign read on the farthest right window?',
 ['bud light',
  'bud light',
  'bud light',
  'bud light',
  'all 2 liters',
  'bud light',
  'bud light',
  'bud light',
  'bud light',
  'bud light'])

In [42]:
for url, question, answers in dataset:
  print(url,question)


https://farm2.staticflickr.com/4/5566811_bc00d504a6_o.jpg what is the brand of this camera?
https://c7.staticflickr.com/5/4136/4920614800_0d28448d76_z.jpg what does the small white text spell?
https://c8.staticflickr.com/4/3519/5721283932_2f54453aca_z.jpg what kind of beer is this?
https://c6.staticflickr.com/6/5590/15223897342_3cbf78f4c9_z.jpg what brand liquor is on the right?
https://c6.staticflickr.com/6/5590/15223897342_3cbf78f4c9_z.jpg how long has the drink on the right been aged?
https://c6.staticflickr.com/1/447/17925009343_f30598134b_z.jpg what number is on the player's jersey?
https://c4.staticflickr.com/9/8446/7993695789_2c52c0c652_z.jpg what is the time?
https://c4.staticflickr.com/9/8446/7993695789_2c52c0c652_z.jpg what brand of watch is that?
https://c4.staticflickr.com/6/5486/10254469926_047ac23028_z.jpg who is at the center of all of this?
https://c7.staticflickr.com/6/5211/5385510861_291d1d1f23_z.jpg who was the photographer?
https://c2.staticflickr.com/3/2686/4333070

In [6]:
    dataset = json.load(open(data))['data']
    dataset = {(data['image_id'], data['question'].lower()): data for data in dataset}
    # results = [json.loads(line) for line in open(result_file)]

    # pred_list = []
    # for result in results:
    #     data = dataset[(result['question_id'], prompt_processor(result['prompt']))]
    #     pred_list.append({
    #         "pred_answer": result['text'],
    #         "gt_answers": data['answers'],
    #     })

    # evaluator = TextVQAAccuracyEvaluator()
    # print('Samples: {}\nAccuracy: {:.2f}%\n'.format(len(pred_list), 100. * evaluator.eval_pred_list(pred_list)))

TypeError: expected str, bytes or os.PathLike object, not dict

In [ ]:
json

In [3]:
!pip install datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00


In [ ]:

_CITATION = """
@inproceedings{singh2019towards,
    title={Towards VQA Models That Can Read},
    author={Singh, Amanpreet and Natarjan, Vivek and Shah, Meet and Jiang, Yu and Chen, Xinlei and Batra, Dhruv and Parikh, Devi and Rohrbach, Marcus},
    booktitle={Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition},
    pages={8317-8326},
    year={2019}
}
"""

_DESCRIPTION = """\
TextVQA requires models to read and reason about text in images to answer questions about them.
Specifically, models need to incorporate a new modality of text present in the images and reason
over it to answer TextVQA questions. TextVQA dataset contains 45,336 questions over 28,408 images
from the OpenImages dataset.
"""

In [46]:
!pip install --upgrade openai -qq
from openai import OpenAI
import os
# API Key를 직접 문자열로 설정
# openai.api_key = "sk-OLKdHXdoDKF6pTLthMqdKNdw7zxJofLEbBmKLPxrZvT3BlbkFJpPUWXHEyDgFwtXhbMGxj0O3o6HTPuDo6D2ngassGkA"
os.environ["OPENAI_API_KEY"] = "sk-v9ZSelBuSsQqyYyfqf9B6pF48R1h3g1OyuEk4Uf3XwT3BlbkFJlDoo59nX36nSKaNVoPjcnmiCxdVwptXb0DvXR39dAA"
client = OpenAI()
# GPT-4 사용 예시


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [78]:
import base64

stored = []
stored2 = []
pred_list = []

responselist2=[]

for img, question, answer in dataset:
  # Then in your request:
  # image_path = f"{img}"
  # encoded_image = encode_image(image_path)

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that responds in Markdown. Help me with explain images!"},

        {"role": "user", "content" : [
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {
                "url": img}
            }]
        }
            ],
      temperature=0.0,
  )
  # pred_list = []
  pred_list.append({
              "pred_answer": (response.choices[0].message.content).lower(),
              "gt_answers": answer,
          })

  evaluator = TextVQAAccuracyEvaluator()
  print(question)
  print(response.choices[0].message.content)
  print(answer)
  responselist2.append(response.choices[0].message.content)
  print('Samples: {}\nAccuracy: {:.2f}%\n'.format(len(pred_list), 100. * evaluator.eval_pred_list(pred_list)))



who was the photographer?
I don't know who the photographer is for this image.
['philippe molitor', 'philippe molitor', 'philippe molitor', 'philippe molitor', 'clardajne', 'phillipe molida', 'l', 'no', 'phillipe meltow', 'philippe molitar']


100%|██████████| 1/1 [00:00<00:00, 803.51it/s]

Samples: 1
Accuracy: 0.00%



are these switches on or off?
The switches in the image are all in the "OFF" position, as indicated by the labels on them.
['off', 'off', 'off', 'off', 'off', 'off', 'off', 'off', 'off', 'off']


100%|██████████| 2/2 [00:00<00:00, 945.41it/s]

...

Samples: 33
Accuracy: 0.00%



what is the largest measurement we can see on this ruler?
The largest measurement visible on the ruler in the image is 50 millimeters (mm). The ruler is marked in increments of 10 mm, with the largest number clearly visible being 50.
['50', ' 50', '50', '50', '50', '50', '50', '50', '50', '50']


100%|██████████| 34/34 [00:00<00:00, 1187.69it/s]

Samples: 34
Accuracy: 0.00%



BadRequestError: Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}

In [76]:
# advanced_self_consistency_approach("You are a helpful assistant that responds in Markdown. Help me with explain images!",[
#             {"type": "text", "text": "Describe the image"},
#             {"type": "image_url", "image_url": {
#                 "url": "https://c8.staticflickr.com/3/2490/4082192766_947a9ba3f8_o.jpg"}
#             }
#         ],client,'gpt-4o-mini')

import base64

stored = []
stored2 = []
pred_list = []

response_list = []

for img, question, answer in dataset:
  # Then in your request:
  # image_path = f"{img}"
  # encoded_image = encode_image(image_path)

  response = advanced_self_consistency_approach("You are a helpful assistant that responds in Markdown. Help me with explain images!",
        [{"type": "text", "text": question},
            {"type": "image_url", "image_url": {
                "url": img}}],client
            ,'gpt-4o-mini')

  # pred_list = []
  pred_list.append({
              "pred_answer": response.lower(),
              "gt_answers": answer,
          })

  # evaluator = TextVQAAccuracyEvaluator()
  evaluator = TextVQAAccuracyEvaluator()
  print(question)
  print(response)
  print(answer)
  response_list.append(response)
  print('Samples: {}\nAccuracy: {:.2f}%\n'.format(len(pred_list), 100. * evaluator.eval_pred_list(pred_list)))



who was the photographer?
I don't know who the photographer is based on the image provided.
['philippe molitor', 'philippe molitor', 'philippe molitor', 'philippe molitor', 'clardajne', 'phillipe molida', 'l', 'no', 'phillipe meltow', 'philippe molitar']


100%|██████████| 1/1 [00:00<00:00, 501.47it/s]

Samples: 1
Accuracy: 0.00%



are these switches on or off?
The switches shown in the image are all in the "OFF" position, as indicated by the markings on them.
['off', 'off', 'off', 'off', 'off', 'off', 'off', 'off', 'off', 'off']


100%|██████████| 2/2 [00:00<00:00, 766.78it/s]

...

Samples: 33
Accuracy: 0.00%



what is the largest measurement we can see on this ruler?
The largest measurement visible on the ruler in the image is 50 units. The ruler is marked in increments of 10, going from 0 to 50.
['50', ' 50', '50', '50', '50', '50', '50', '50', '50', '50']


100%|██████████| 34/34 [00:00<00:00, 860.31it/s]

Samples: 34
Accuracy: 0.00%



BadRequestError: Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}

In [93]:
for i,t in zip(responselist2,response_list):
  print(f"basemodel_result : {i}  \nsc result :{t}")
  print(f"동일한 응답인가? : {i ==t}\n")

basemodel_result : I don't know who the photographer is for this image.  
sc result :I don't know who the photographer is based on the image provided.
동일한 응답인가? : False

basemodel_result : The switches in the image are all in the "OFF" position, as indicated by the labels on them.  
sc result :The switches shown in the image are all in the "OFF" position, as indicated by the markings on them.
동일한 응답인가? : False

basemodel_result : The candy bar at the bottom of the image is a **Hershey's** chocolate bar.  
sc result :The candy bar at the bottom of the image is a **Hershey's chocolate bar**.
동일한 응답인가? : False

basemodel_result : The light sign on the farthest right window reads: "All 2-Lite 7 for $3.49 or 9 for $4.99."  
sc result :The light sign on the farthest right window reads "All 2-Liter 2 for $3.49".
동일한 응답인가? : False

basemodel_result : In the image, the price for a can of Skoal is listed as **$4.52**.  
sc result :In the image, the price for a can of Skoal is displayed as $4.52.

In [ ]:
import base64

stored = []
stored2 = []
pred_list = []



for img, question, answer in dataset:
  # Then in your request:
  # image_path = f"{img}"
  # encoded_image = encode_image(image_path)

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that responds in Markdown. Help me with explain images!"},

        {"role": "user", "content" : [
            {"type": "text", "text": question},
            {"type": "image_url", "image_url": {
                "url": img}
            }]
        }
            ],
      temperature=0.0,
  )
  # pred_list = []
  pred_list.append({
              "pred_answer": (response.choices[0].message.content).lower(),
              "gt_answers": answer,
          })

  evaluator = TextVQAAccuracyEvaluator()
  print(question)
  print(response.choices[0].message.content)
  print(answer)
  print('Samples: {}\nAccuracy: {:.2f}%\n'.format(len(pred_list), 100. * evaluator.eval_pred_list(pred_list)))



In [61]:
import json

with open('val.json', 'r') as f:
  data = json.load(f)

# data = data['data'][9:]
dataset = [(data['flickr_300k_url'], data['question'].lower(), data['answers']) for data in data['data'][9:]]


In [51]:
import os
import argparse
import json
import re

#  TextVQAAccuracyEvaluator


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data-file', type=str)
    parser.add_argument('--result-file', type=str)
    parser.add_argument('--result-dir', type=str)
    return parser.parse_args()


def prompt_processor(prompt):
    if prompt.startswith('OCR tokens: '):
        pattern = r"Question: (.*?) Short answer:"
        match = re.search(pattern, prompt, re.DOTALL)
        question = match.group(1)
    elif 'Reference OCR token: ' in prompt and len(prompt.split('\n')) == 3:
        if prompt.startswith('Reference OCR token:'):
            question = prompt.split('\n')[1]
        else:
            question = prompt.split('\n')[0]
    elif len(prompt.split('\n')) == 2:
        question = prompt.split('\n')[0]
    else:
        assert False

    return question.lower()


def eval_single(data_file, result_file):
    experiment_name = os.path.splitext(os.path.basename(result_file))[0]
    print(experiment_name)
    dataset = json.load(open(data_file))['data']
    dataset = {(data['image_id'], data['question'].lower()): data for data in dataset}
    results = [json.loads(line) for line in open(result_file)]

    pred_list = []
    for result in results:
        data = dataset[(result['question_id'], prompt_processor(result['prompt']))]
        pred_list.append({
            "pred_answer": result['text'],
            "gt_answers": data['answers'],
        })

    evaluator = TextVQAAccuracyEvaluator()
    print('Samples: {}\nAccuracy: {:.2f}%\n'.format(len(pred_list), 100. * evaluator.eval_pred_list(pred_list)))


if __name__ == "__main__":
    args = get_args()

    if args.result_file is not None:
        eval_single(args.data_file, args.result_file)

    if args.result_dir is not None:
        for result_file in sorted(os.listdir(args.result_dir)):
            if not result_file.endswith('.jsonl'):
                print(f'Skipping {result_file}')
                continue
            eval_single(args.data_file, os.path.join(args.result_dir, result_file))

usage: colab_kernel_launcher.py [-h] [--data-file DATA_FILE] [--result-file RESULT_FILE]
                                [--result-dir RESULT_DIR]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-ba619dcb-948c-41c5-9c67-c02cff18c12e.json


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [72]:
import logging
from typing import List, Dict
from difflib import SequenceMatcher

logger = logging.getLogger(__name__)

class AdvancedSelfConsistency:
    def __init__(self, client, model: str,  num_samples: int = 5, similarity_threshold: float = 0.8):
        self.client = client
        self.model = model
        self.num_samples = num_samples
        self.similarity_threshold = similarity_threshold
        self.self_consistency_completion_tokens = 0

    def generate_responses(self, system_prompt: str, user_prompt) -> List[str]:
        responses = []
        for _ in range(self.num_samples):
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=1,
                max_tokens=4096
            )
            self.self_consistency_completion_tokens += response.usage.completion_tokens
            responses.append(response.choices[0].message.content)
        return responses

    def calculate_similarity(self, a: str, b: str) -> float:
        return SequenceMatcher(None, a, b).ratio()

    def cluster_similar_responses(self, responses: List[str]) -> List[List[str]]:
        clusters = []
        for response in responses:
            added_to_cluster = False
            for cluster in clusters:
                if self.calculate_similarity(response, cluster[0]) >= self.similarity_threshold:
                    cluster.append(response)
                    added_to_cluster = True
                    break
            if not added_to_cluster:
                clusters.append([response])
        return clusters

    def aggregate_results(self, responses: List[str]) -> Dict[str, any]:
        final_answers = responses
        clusters = self.cluster_similar_responses(final_answers)

        cluster_info = []
        for cluster in clusters:
            cluster_info.append({
                "answer": cluster[0],
                "frequency": len(cluster),
                "variants": cluster
            })

        cluster_info.sort(key=lambda x: x['frequency'], reverse=True)

        return {
            "clusters": cluster_info,
            "total_responses": len(responses),
            "num_unique_clusters": len(clusters)
        }

    def evaluate(self, system_prompt: str, user_prompt) -> Dict[str, any]:
        responses = self.generate_responses(system_prompt, user_prompt)
        aggregated_result = self.aggregate_results(responses)

        return {
            "individual_responses": responses,
            "aggregated_result": aggregated_result
        }

def advanced_self_consistency_approach(system_prompt: str, initial_query, client, model: str) -> str:
    self_consistency = AdvancedSelfConsistency(client, model)
    result = self_consistency.evaluate(system_prompt, initial_query)

    logger.info("Advanced Self-Consistency Results:")
    logger.info(f"Total responses: {result['aggregated_result']['total_responses']}")
    logger.info(f"Number of unique clusters: {result['aggregated_result']['num_unique_clusters']}")
    for i, cluster in enumerate(result['aggregated_result']['clusters'], 1):
        logger.debug(f"\nCluster {i}:")
        logger.debug(f"  Representative answer: {cluster['answer']}")
        logger.debug(f"  Frequency: {cluster['frequency']}")
        logger.debug(f"  Variants: {cluster['variants']}")

    if result['aggregated_result']['clusters']:
        return result['aggregated_result']['clusters'][0]['answer'] #self_consistency.self_consistency_completion_tokens
    else:
        return "No consistent answer found." #self_consistency.self_consistency_completion_tokens